In [24]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'fairbet_django.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [25]:
from order_placed.models import *
from payment.models import *
from django_pandas.io import read_frame
from django.db.models import F, Q, When, Value, Case, Sum

In [26]:
winningTeam = "England"
matchName = "England vs India"

In [27]:
betting_instance = Betting.objects.filter(
        match = matchName
    ).update(
        loss_profit = Case(
            When(
                status = "B",
                bet_on_team = winningTeam,
                winning_team = winningTeam,
                then = F('amount') * F("odds") - F('amount')),
            When(
                ~Q(bet_on_team = winningTeam),
                status = "B",
                winning_team = winningTeam,
                then = -F('amount')),
             When(
                status = "L",
                bet_on_team = winningTeam,
                winning_team = winningTeam,
                then = F('amount') * - F("odds") + F('amount')),
             When(
                 ~Q(bet_on_team = winningTeam),
                status = "L",
                winning_team = winningTeam,
                then = F('amount')),
            default=F("loss_profit")
            )
        )

In [141]:
get_wallet = read_frame(Betting.objects.all())

In [142]:
get_wallet

,id,user,amount,bet_on_team,status,odds,winning_team,loss_profit,match,created_at,updated_at
0,16,fairbet,1000.0,England,BACK,1.50,England,500.0,England vs India,2022-07-06 06:20:16.207060+00:00,2022-07-06 10:31:52.837866+00:00
1,17,fairbet,1000.0,England,LAY,1.52,England,-520.0,England vs India,2022-07-06 06:23:12.963245+00:00,2022-07-06 10:31:56.493455+00:00
2,21,fairbet,1000.0,India,BACK,1.55,England,-1000.0,England vs India,2022-07-06 10:33:05.626427+00:00,2022-07-06 10:33:31.403796+00:00
3,22,fairbet,1000.0,India,LAY,1.56,England,1000.0,England vs India,2022-07-06 10:33:16.385601+00:00,2022-07-06 10:33:44.568271+00:00
4,23,Minkyaa6,2000.0,India,BACK,1.50,England,-2000.0,England vs India,2022-07-07 06:08:40.586928+00:00,2022-07-07 06:12:14.663329+00:00
5,24,Minkyaa6,1500.0,India,LAY,1.50,England,1500.0,England vs India,2022-07-07 06:08:58.262509+00:00,2022-07-07 06:12:19.332599+00:00
6,25,Minkyaa6,1800.0,England,LAY,1.50,England,-900.0,England vs India,2022-07-07 06:11:54.038143+00:00,2022-07-07 06:12:23.678379+00:00
7,26,Minkyaa6,2800.0,England,BACK,1.50,England,1400.0,England vs India,2022-07-07 06:12:01.840215+00:00,2022-07-07 06:12:27.971871+00:00


In [143]:
get_wallet['total_loss_or_profit'] = get_wallet.groupby('user')['loss_profit'].transform(lambda x : sum(x))

In [144]:
get_wallet.drop_duplicates(subset=['user'])

,id,user,amount,bet_on_team,status,odds,winning_team,loss_profit,match,created_at,updated_at,total_loss_or_profit
0,16,fairbet,1000.0,England,BACK,1.5,England,500.0,England vs India,2022-07-06 06:20:16.207060+00:00,2022-07-06 10:31:52.837866+00:00,-20.0
4,23,Minkyaa6,2000.0,India,BACK,1.5,England,-2000.0,England vs India,2022-07-07 06:08:40.586928+00:00,2022-07-07 06:12:14.663329+00:00,0.0


In [145]:
Betting.objects.all().annotate(total_sum=Sum("loss_profit"))

<BulkUpdateOrCreateQuerySet [<Betting: Betting object (16)>, <Betting: Betting object (17)>, <Betting: Betting object (21)>, <Betting: Betting object (22)>, <Betting: Betting object (23)>, <Betting: Betting object (24)>, <Betting: Betting object (25)>, <Betting: Betting object (26)>]>

In [146]:
df = read_frame(Betting.objects.values("user__id","user_id","user_id__wallet__amount").annotate(loss_or_profit = Sum("loss_profit")))

In [147]:
df['total'] = df['user_id__wallet__amount'] + df['loss_or_profit']

In [148]:
df

,user__id,user_id,user_id__wallet__amount,loss_or_profit,total
0,1,fairbet,9500.0,-20.0,9480.0
1,5,Minkyaa6,10000.0,0.0,10000.0


In [149]:
Betting.objects.values("user__id","user_id","user_id__wallet__amount").annotate(total_amount = Sum("loss_profit"))

<BulkUpdateOrCreateQuerySet [{'user__id': 1, 'user_id': 1, 'user_id__wallet__amount': 9500.0, 'total_amount': -20.0}, {'user__id': 5, 'user_id': 5, 'user_id__wallet__amount': 10000.0, 'total_amount': 0.0}]>

In [150]:
Betting.objects.filter(match="England vs India").values("user__id","user_id","user_id__wallet__amount").annotate(loss_or_profit = Sum("loss_profit"))

<BulkUpdateOrCreateQuerySet [{'user__id': 1, 'user_id': 1, 'user_id__wallet__amount': 9500.0, 'loss_or_profit': -20.0}, {'user__id': 5, 'user_id': 5, 'user_id__wallet__amount': 10000.0, 'loss_or_profit': 0.0}]>

In [151]:
 df = read_frame(Betting.objects.filter(match=matchName,winning_team=winningTeam).values("user__id","user_id","user_id__wallet__amount").annotate(loss_or_profit = Sum("loss_profit")))

In [152]:
df['total'] = df['user_id__wallet__amount'] + df['loss_or_profit']

In [153]:
wallet_dict = df.transpose().to_dict()

In [157]:
wallet_dict

{0: {'user__id': 1,
  'user_id': 'fairbet',
  'user_id__wallet__amount': 9500.0,
  'loss_or_profit': -20.0,
  'total': 9480.0},
 1: {'user__id': 5,
  'user_id': 'Minkyaa6',
  'user_id__wallet__amount': 10000.0,
  'loss_or_profit': 0.0,
  'total': 10000.0}}

In [156]:
update_wallet_instance = [Wallet(
    user_id = value.get("user__id",""),
    amount = value.get("total",""),
    ) for key,value in wallet_dict.items()]
Wallet.objects.bulk_update_or_create(update_wallet_instance,['user_id','amount'],match_field=['user_id'])

AttributeError: 'Manager' object has no attribute 'bulk_update_or_create'

In [95]:
bet = read_frame(Betting.objects.filter(match=matchName).values("user__id","user","amount","bet_on_team","status","odds","winning_team","loss_profit","match"))

In [ ]:
update_wallet_instance = [Wallet(
    user_id = value.get("vendor_id",""),
    amount = value.get("product_id",""),
    ) for key,value in colour_dict.items()]
Wallet.objects.bulk_update_or_create(update_wallet_instance,['user_id','amount'],match_field=['user_id'])

In [131]:
bettings = Betting.objects.filter(loss_profit__gt=0)

In [132]:
bettings

<BulkUpdateOrCreateQuerySet [<Betting: Betting object (7)>, <Betting: Betting object (10)>]>

In [40]:
df = read_frame(Betting.objects.filter(match=matchName).values("user__id","user","amount","bet_on_team","status","odds","winning_team","loss_profit","match"))

In [48]:
new_df = df.groupby('user')['loss_profit'].sum()

In [54]:
df.groupby('user')['loss_profit'].transform('sum')

0   -20.0
1   -20.0
2   -20.0
3   -20.0
Name: loss_profit, dtype: float64

In [49]:
new_df.reset_index(drop=True,inplace=True)

In [50]:
new_df

0   -20.0
Name: loss_profit, dtype: float64

In [38]:
bett = Betting.objects.filter(user_id=1).aggregate(Sum(F('loss_profit')))

In [39]:
bett

{'loss_profit__sum': -20.0}

In [25]:
wallet_df = read_frame(Wallet.objects)

,id,user,amount,bet_on_team,status,odds,winning_team,loss_profit,match,created_at,updated_at
0,16,fairbet,1000.0,England,BACK,1.50,England,500.0,England vs India,2022-07-06 06:20:16.207060+00:00,2022-07-06 10:31:52.837866+00:00
1,17,fairbet,1000.0,England,LAY,1.52,England,-520.0,England vs India,2022-07-06 06:23:12.963245+00:00,2022-07-06 10:31:56.493455+00:00
2,21,fairbet,1000.0,India,BACK,1.55,England,-1000.0,England vs India,2022-07-06 10:33:05.626427+00:00,2022-07-06 10:33:31.403796+00:00
3,22,fairbet,1000.0,India,LAY,1.56,England,1000.0,England vs India,2022-07-06 10:33:16.385601+00:00,2022-07-06 10:33:44.568271+00:00


In [68]:
user_df = read_frame(Betting.objects.all().values("user__id","user","amount","bet_on_team","status","odds","winning_team","loss_profit","match"))

In [69]:
user_df

,user__id,user,amount,bet_on_team,status,odds,winning_team,loss_profit,match
0,1,fairbet,1000.0,England,BACK,1.50,England,500.0,England vs India
1,1,fairbet,1000.0,England,LAY,1.52,England,-520.0,England vs India
2,1,fairbet,1000.0,India,BACK,1.55,England,-1000.0,England vs India
3,1,fairbet,1000.0,India,LAY,1.56,England,1000.0,England vs India


In [158]:
#Subquery

In [184]:
from django.db.models import OuterRef, Subquery

In [185]:
# Task.objects.annotate(
#     deal_id=Subquery(
#         DealTask.objects.filter(
#             task_id=OuterRef('id')
#         ).order_by('deadline').values('deal_id')[:1]
#     ),
#     deadline=Subquery(
#         DealTask.objects.filter(
#             task_id=OuterRef('id')
#         ).order_by('deadline').values('deadline')[:1]
#     )
# )

In [186]:
Betting.objects.filter(match=matchName,winning_team=winningTeam).values("user__id","user_id","user_id__wallet__amount").annotate(loss_or_profit = Sum("loss_profit"))

<BulkUpdateOrCreateQuerySet [{'user__id': 1, 'user_id': 1, 'user_id__wallet__amount': 9480.0, 'loss_or_profit': -20.0}, {'user__id': 5, 'user_id': 5, 'user_id__wallet__amount': 10000.0, 'loss_or_profit': 0.0}]>

In [417]:
# Wallet.objects.update(
#     amount=F('amount') + Subquery(
#         Betting.objects.filter(
#             user_id=OuterRef('user_id'))
#         .values("user__id").annotate(total_amount=Sum('loss_profit').values('total_amount'))
#     )
# )

AttributeError: 'Sum' object has no attribute 'values'

In [424]:
Betting.objects.filter(user_id=1).values("user__id").annotate(total_loss_profit=Sum('loss_profit')).values("total_loss_profit")

<BulkUpdateOrCreateQuerySet [{'total_loss_profit': -20.0}]>

In [394]:
Betting.objects.filter(user_id=1).values("loss_profit")

<BulkUpdateOrCreateQuerySet [{'loss_profit': 500.0}, {'loss_profit': -520.0}, {'loss_profit': -1000.0}, {'loss_profit': 1000.0}]>

In [493]:
winningTeam = "England"
matchName = "England vs India"

In [494]:
subquery = Betting.objects.filter(
        user_id=OuterRef('user_id'),
        match = matchName,
        winning_team = winningTeam
    ).values(
        'user__id'
    ).annotate(
        total_spend=Sum('loss_profit')
    ).values(
        'total_spend'
    )

In [495]:
betting_user_list = list(Betting.objects.filter(match = matchName,winning_team = winningTeam).values_list("user_id",flat=True))

In [496]:
list(take_list)

[1, 5]

In [480]:
Wallet.objects.filter(user_id__in=list(take_list)).update(amount=F('amount')+Subquery(subquery))

2

In [410]:
Betting.objects.all().values("user__id").annotate(total_amount=Sum('loss_profit')).values("total_amount")

<BulkUpdateOrCreateQuerySet [{'total_amount': -20.0}, {'total_amount': 0.0}]>

In [414]:
Betting.objects.all().values("user__id").annotate(total_spend = Sum(F('loss_profit')))

<BulkUpdateOrCreateQuerySet [{'user__id': 1, 'total_spend': -20.0}, {'user__id': 5, 'total_spend': 0.0}]>

In [389]:
# Wallet.objects.update(
#     amount=F('amount') + Subquery(
#         Betting.objects.filter(
#             user_id=OuterRef('user_id'))
#         .values("loss_profit").aggregate(Sum("loss_profit"))['loss_profit__sum']
#         )
# )

ValueError: This queryset contains a reference to an outer query and may only be used in a subquery.

In [333]:
Betting.objects.filter(user_id=1).values("loss_profit").aggregate(Sum("loss_profit"))

{'loss_profit__sum': -20.0}

In [342]:
comments = Betting.objects.filter(user_id=OuterRef('user_id')).order_by().values('loss_profit')

In [344]:
# comments

In [ ]:
hero_qs = Wallet.objects.filter(
    user_id=OuterRef("user_id")
).order_by("-created")
Betting.objects.all().annotate(
    most_benevolent_hero=Subquery(
        hero_qs.values('amount')[:1]
    )
)

In [376]:
bet = Betting.objects.filter(user_id=5).values("user__id","loss_profit").aggregate(loss_profit=Sum(F("loss_profit")))['loss_profit']

In [377]:
# Wallet.objects.filter()

<QuerySet [<Wallet: fairbet>, <Wallet: Minkyaa6>]>

In [386]:
Wallet.objects.filter(
    user_id=5
).order_by("-created").values('amount')

<QuerySet [{'amount': 6000.0}]>

In [330]:
# Betting.objects.filter(user_id=OuterRef('user_id')).values("loss_profit")

In [231]:
Betting.objects.all().values("user__id","user_id","user_id__wallet__amount").annotate(loss_profit = Sum("loss_profit"))

<BulkUpdateOrCreateQuerySet [{'user__id': 1, 'user_id': 1, 'user_id__wallet__amount': 9500.0, 'loss_profit': -20.0}, {'user__id': 5, 'user_id': 5, 'user_id__wallet__amount': 10000.0, 'loss_profit': 0.0}]>

In [210]:
Betting.objects.all().aggregate(Sum(F("loss_profit")))

{'loss_profit__sum': -20.0}

In [211]:
Betting.objects.filter(match=matchName,winning_team=winningTeam).values("user__id","user_id","user_id__wallet__amount").annotate(loss_or_profit = Sum("loss_profit"))

<BulkUpdateOrCreateQuerySet [{'user__id': 1, 'user_id': 1, 'user_id__wallet__amount': 500.0, 'loss_or_profit': -20.0}, {'user__id': 5, 'user_id': 5, 'user_id__wallet__amount': -2000.0, 'loss_or_profit': 0.0}]>

In [441]:
get_json = [{"tags":"[1,2]"}]

In [442]:
import json

In [443]:
dump = json.dumps(get_json)

In [444]:
dump

'[{"tags": "[1,2]"}]'

In [438]:
json.loads(dump)

[{'tags': [1, 2]}]

In [449]:
Betting.objects.filter(
        user_id=1,
        match = matchName,
        winning_team = winningTeam
    ).values(
        'user__id'
    ).annotate(
        total_spend=Sum('loss_profit')
    ).values(
        'total_spend'
    )

<BulkUpdateOrCreateQuerySet [{'total_spend': -20.0}]>

In [12]:
bet = Betting.objects.all().values("user__id").annotate(loss_profit=Sum(F("loss_profit"))).values("loss_profit")

In [13]:
bet

<BulkUpdateOrCreateQuerySet [{'loss_profit': -20.0}, {'loss_profit': 0.0}, {'loss_profit': -1800.0}]>

In [ ]:
#Football api

In [1]:
import requests

In [ ]:
##Sample payload
# url = "https://v3.football.api-sports.io/{endpoint}"

# payload={}
# headers = {
#   'x-rapidapi-key': 'XxXxXxXxXxXxXxXxXxXxXxXx',
#   'x-rapidapi-host': 'v3.football.api-sports.io'
# }

# response = requests.request("GET", url, headers=headers, data=payload)

# print(response.text)

In [18]:
import http.client
import json
import pandas as pd
conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "344b4657332d395aa1a19b3027f9b8f2"
    }

conn.request("GET", "/odds/bookmakers", headers=headers)

res = conn.getresponse()
data = res.read()

data = json.loads(data.decode("utf-8"))
df = pd.DataFrame([data])
df

,get,parameters,errors,results,paging,response
0,odds/bookmakers,[],[],28,"{'current': 1, 'total': 1}","[{'id': 1, 'name': '10Bet'}, {'id': 2, 'name':..."


In [20]:
df['response'].iloc[0]

[{'id': 1, 'name': '10Bet'},
 {'id': 2, 'name': 'Marathonbet'},
 {'id': 3, 'name': 'Betfair'},
 {'id': 4, 'name': 'Pinnacle'},
 {'id': 5, 'name': 'SBO'},
 {'id': 6, 'name': 'Bwin'},
 {'id': 7, 'name': 'William Hill'},
 {'id': 8, 'name': 'Bet365'},
 {'id': 9, 'name': 'Dafabet'},
 {'id': 10, 'name': 'Ladbrokes'},
 {'id': 11, 'name': '1xBet'},
 {'id': 12, 'name': 'BetFred'},
 {'id': 13, 'name': '188Bet'},
 {'id': 15, 'name': 'Interwetten'},
 {'id': 16, 'name': 'Unibet'},
 {'id': 17, 'name': '5Dimes'},
 {'id': 18, 'name': 'Intertops'},
 {'id': 19, 'name': 'Bovada'},
 {'id': 20, 'name': 'Betcris'},
 {'id': 21, 'name': '888Sport'},
 {'id': 22, 'name': 'Tipico'},
 {'id': 23, 'name': 'Sportingbet'},
 {'id': 24, 'name': 'Betway'},
 {'id': 25, 'name': 'Expekt'},
 {'id': 26, 'name': 'Betsson'},
 {'id': 27, 'name': 'NordicBet'},
 {'id': 28, 'name': 'ComeOn'},
 {'id': 30, 'name': 'Netbet'}]

In [32]:
aggregate_betting_amount = Betting.objects.filter(user_id=1).values("user_id").annotate(get_amount = Sum("amount")).values('get_amount')

In [35]:
aggregate_betting_amount[0]['get_amount']

4000.0